In [55]:
# Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from typing import TypedDict, List, Dict, Optional
import os
import json
import dotenv

# LLM and embedding related imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, START, END

# Evaluation related imports
from ragas.metrics import AnswerAccuracy, ContextRelevance, ResponseGroundedness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

# Load environment variables
dotenv.load_dotenv()


True

In [56]:
# Model and Vector Store Setup

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = InMemoryVectorStore(embeddings)

In [44]:
# Document Loading

docs = []
data_folder = "data"

if os.path.exists(data_folder) and os.path.isdir(data_folder):
    for filename in os.listdir(data_folder):
        if filename.endswith(".pdf"):
            file_path = os.path.join(data_folder, filename)
            try:
                loader = PyPDFLoader(file_path)
                file_docs = loader.load()
                docs.extend(file_docs)
                print(f"Loaded {len(file_docs)} documents from {filename}")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
else:
    print(f"Folder {data_folder} does not exist or is not a directory")

print(f"Total documents loaded: {len(docs)}")

Loaded 9 documents from Anthropic Wikipedia.pdf
Loaded 11 documents from Mistral AI.pdf
Loaded 7 documents from XAI PDF Wikipedia.pdf
Loaded 40 documents from Nvidia PDF Wikipedia.pdf
Loaded 55 documents from OpenAI Wikipedia.pdf
Loaded 19 documents from DeepSeek Wikipedia.pdf
Loaded 5 documents from Hugging Face.pdf
Loaded 24 documents from DeepMind Wikipedia.pdf
Total documents loaded: 170


In [57]:
# Cell 4: Document Processing and Vectorization

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

print("Splitting documents into chunks...")
splitted_docs = text_splitter.split_documents(docs)
print(f"Splitted documents into: {len(splitted_docs)} chunks")

print("Adding documents to vector store...")
_ = vectorstore.add_documents(splitted_docs)
print("Documents added to vector store")


Splitting documents into chunks...
Splitted documents into: 729 chunks
Adding documents to vector store...
Documents added to vector store


In [58]:
# RAG Pipeline Implementation with LangGraph

class State(TypedDict):
    query: str
    context: List[Document]
    answer: str
    
def retrieve(state: State):
    print(f"🔍 Retrieving documents for: {state['query']}")
    retriever = vectorstore.similarity_search(query=state["query"], k=2)
    return {"context": retriever}

def generate_answer(state: State):
    print("💬 Generating answer...")
    
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    
    prompt = f"""
    You are a helpful AI assistant. Answer the question based only on the context provided. 
    If the context doesn't contain the information needed to answer the question, say "I don't have enough information to answer this question."
    
    Question: {state["query"]}
    Context: {docs_content}
    
    Provide a concise and accurate answer based solely on the information in the context:
    """
    
    response = llm.invoke(prompt)
    return {"answer": response.content}

# Build RAG pipeline graph
graph_builder = StateGraph(State)
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("generate_answer", generate_answer)

# Connect nodes in sequence
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate_answer")
graph_builder.add_edge("generate_answer", END)

# Compile the graph
graph = graph_builder.compile()
print("✓ RAG pipeline ready")


✓ RAG pipeline ready


In [59]:
graph.invoke({"query": "what is deep mind?"})

🔍 Retrieving documents for: what is deep mind?
💬 Generating answer...


{'query': 'what is deep mind?',
 'context': [Document(id='af0222c0-63be-4f11-87e3-9d0e2ae90819', metadata={'producer': 'Skia/PDF m135', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/135.0.0.0 Safari/537.36', 'creationdate': '2025-05-11T18:49:21+00:00', 'title': 'Google DeepMind - Wikipedia', 'moddate': '2025-05-11T18:49:21+00:00', 'source': 'data/DeepMind Wikipedia.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content="laboratory which serves as a subsidiary of Alphabet\nInc. Founded in the UK in 2010, it was acquired by\nGoogle in 2014[8] and merged with Google AI's\nGoogle Brain division to become Google DeepMind\nin April 2023. The company is headquartered in\nLondon, with research centres in the United States,\nCanada,[9] France,[10] Germany and Switzerland.\nDeepMind introduced neural Turing machines (neural\nnetworks that can access external memory like a\nconventional Turing machine),[11] resulting in a\ncomputer

In [64]:
# Cell 7: Load and Explore Test Dataset
print("Loading test dataset...")
test_data = pd.read_csv("test-set/synthetic-test-set.csv")
print(f"Loaded {len(test_data)} test examples with query - ground_truth pairs")
test_data.head(3)

Loading test dataset...
Loaded 42 test examples with query - ground_truth pairs


,query,ground_truth
0,When was OpenAI founded?,"December 8, 2015"
1,Who are the founders of OpenAI?,"John Schulman, Elon Musk, Ilya Sutskever, Sam ..."
2,What is OpenAI's mission?,To ensure that AGI benefits all of humanity.


In [65]:
# Processing of Test Questions

print(f"Processing {len(test_data)} test questions...")
results = []

for index, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Generating answers"):
    query = row['query']
    ground_truth = row['ground_truth']
    
    result = graph.invoke({"query": query})
    
    results.append({
        "query": query,
        "ground_truth": ground_truth,
        "response": result["answer"],
        "contexts": [doc.page_content for doc in result["context"]]
    })
    
   

# Save final results to JSON file
with open("test_results.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"✓ Successfully generated answers for {len(results)} questions")
print(f"Results saved to 'test_results.json'")

Processing 42 test questions...


Generating answers:   0%|          | 0/42 [00:00<?, ?it/s]

🔍 Retrieving documents for: When was OpenAI founded?
💬 Generating answer...
🔍 Retrieving documents for: Who are the founders of OpenAI?
💬 Generating answer...
🔍 Retrieving documents for: What is OpenAI's mission?
💬 Generating answer...
🔍 Retrieving documents for: What are some key products developed by OpenAI?
💬 Generating answer...
🔍 Retrieving documents for: Where is OpenAI headquartered?
💬 Generating answer...
🔍 Retrieving documents for: Who is OpenAI's current CEO?
💬 Generating answer...
🔍 Retrieving documents for: Which major company invested $13 billion in OpenAI?
💬 Generating answer...
🔍 Retrieving documents for: What is the estimated revenue of OpenAI in 2024?
💬 Generating answer...
🔍 Retrieving documents for: Why did Elon Musk leave OpenAI?
💬 Generating answer...
🔍 Retrieving documents for: What is the name of OpenAI’s AI chatbot released in November 2022?
💬 Generating answer...
🔍 Retrieving documents for: When was Anthropic founded?
💬 Generating answer...
🔍 Retrieving documen

In [67]:
# Prepare Evaluation Dataset for RAGAS
print("Preparing evaluation dataset...")
samples = []

# Convert results to RAGAS format
for item in tqdm(results, desc="Converting to RAGAS format"):
    sample = SingleTurnSample(
        user_input=item["query"],
        response=item["response"],
        reference=item["ground_truth"],
        retrieved_contexts=item["contexts"]
    )
    samples.append(sample)

# Create evaluation dataset
dataset = EvaluationDataset(samples=samples)
print(f"✓ Prepared evaluation dataset with {len(samples)} samples")

Preparing evaluation dataset...


Converting to RAGAS format:   0%|          | 0/42 [00:00<?, ?it/s]

✓ Prepared evaluation dataset with 42 samples


In [69]:
# RAGAS Evaluation with Visualizations

# Initialize evaluation metrics
metrics = [
    AnswerAccuracy(),   # Measures correctness against reference answer
    ContextRelevance(), # Measures relevance of retrieved context
    ResponseGroundedness() # Measures if response is grounded in context
]

print("Running RAGAS evaluation...")
# Wrap LLM for RAGAS
ragas_llm = LangchainLLMWrapper(llm)
# Run evaluation
result = evaluate(
    dataset=dataset,
    metrics=metrics,
    llm=ragas_llm
)
print("✓ Evaluation complete")
print(result)
result.to_pandas()

Running RAGAS evaluation...


Evaluating:   0%|          | 0/126 [00:00<?, ?it/s]

✓ Evaluation complete
{'nv_accuracy': 0.7321, 'nv_context_relevance': 0.9286, 'nv_response_groundedness': 0.9524}


,user_input,retrieved_contexts,response,reference,nv_accuracy,nv_context_relevance,nv_response_groundedness
0,When was OpenAI founded?,[OpenAI\nCompany type Private\nIndustry Artifi...,"OpenAI was founded on December 8, 2015.","December 8, 2015",1.00,1.0,1.0
1,Who are the founders of OpenAI?,[Website openai.com (https://openai.\ncom/)\nF...,"The founders of OpenAI are Sam Altman, Elon Mu...","John Schulman, Elon Musk, Ilya Sutskever, Sam ...",0.50,1.0,1.0
2,What is OpenAI's mission?,"[to develop ""safe and beneficial"" artificial g...","OpenAI's mission is to develop ""safe and benef...",To ensure that AGI benefits all of humanity.,0.25,1.0,1.0
3,What are some key products developed by OpenAI?,"[to develop ""safe and beneficial"" artificial g...",Some key products developed by OpenAI include ...,"ChatGPT, GPT models, DALL·E, Codex, Sora",0.50,1.0,1.0
4,Where is OpenAI headquartered?,[OpenAI\nCompany type Private\nIndustry Artifi...,"OpenAI is headquartered at 1455 3rd Street, Sa...","San Francisco, California",0.75,1.0,1.0
5,Who is OpenAI's current CEO?,[private technology deal on record. The financ...,OpenAI's current CEO is Sam Altman.,Sam Altman,1.00,1.0,1.0
6,Which major company invested $13 billion in Op...,"[In December 2022, OpenAI received widespread ...",Microsoft invested $10 billion in OpenAI.,Microsoft,0.25,0.5,1.0
7,What is the estimated revenue of OpenAI in 2024?,"[In December 2022, OpenAI received widespread ...",The estimated revenue of OpenAI in 2024 is $1 ...,$3.7 billion,0.00,1.0,1.0
8,Why did Elon Musk leave OpenAI?,"[32. Claburn, Thomas. ""Elon Musk-backed OpenAI...",Elon Musk left the board of an AI safety group...,Due to a potential future conflict of interest...,0.50,0.5,1.0
9,What is the name of OpenAI’s AI chatbot releas...,[The release of ChatGPT was a major\nevent in ...,The name of OpenAI’s AI chatbot released in No...,ChatGPT,1.00,1.0,1.0
